### Census Trade Data

September 30, 2020

Brian Dew

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [2]:
# Retrieve data from Census API
d = {}

for flow in ['imports', 'exports']:
    base = f'https://api.census.gov/data/timeseries/intltrade/{flow}/hs'
    params = ['CTY_CODE', 'CTY_NAME', 'ALL_VAL_MO']
    if flow == 'imports':
        params[2] = 'GEN_VAL_MO'
    
    param = ','.join(params)
    url = f'{base}?get={param}&key={census_key}&time=from+1992-01'

    r = requests.get(url).json()

    d[flow] = r

In [3]:
i = d['imports']
df = pd.DataFrame(i[1:], columns=i[0])
df = df[(df['CTY_CODE'].str[0] !='0') & 
        (df['CTY_CODE'].str[1] != 'X')]
df['Name'] = df['CTY_NAME'].str.title()
df['Date'] = pd.to_datetime(df['time'], format='%Y-%m')
imp = (df.rename({'GEN_VAL_MO': 'Imports'}, axis=1)
         .set_index(['Name', 'Date']).Imports.astype(float))

e = d['exports']
df = pd.DataFrame(e[1:], columns=e[0])
df = df[(df['CTY_CODE'].str[0] !='0') & 
        (df['CTY_CODE'].str[1] != 'X')]
df['Name'] = df['CTY_NAME'].str.title()
df['Date'] = pd.to_datetime(df['time'], format='%Y-%m')
exp = (df.rename({'ALL_VAL_MO': 'Exports'}, axis=1)
         .set_index(['Name', 'Date']).Exports.astype(float))

data = ((pd.DataFrame([imp, exp]).T / 1000000)
           .rename({'Korea, South': 'South Korea',
                    'Total For All Countries': 'Total, All Countries'}))

data.to_csv(data_dir / 'trade_master.csv')

data['Total'] = data['Imports'] + data['Exports']

dt = data.xs('Canada').index[-1]
ltdate = dtxt(dt)['mon1']
index = data.xs(dt, level=1).Total.sort_values(ascending=False).iloc[:26].index

final = data.loc[(index, dt),:].astype(int).droplevel('Date')
dt2 = dt - pd.DateOffset(months = 12)
data2 = data.loc[(index, dt2),:].astype(int).droplevel('Date')
data2.columns = [f'{k} ' for k in final.keys()]
final = final.join(data2).sort_values('Total', ascending=False)
final.index.name = ''

(final.applymap('{:,.0f}'.format)
      .to_csv(data_dir / 'trade_partner.tex', sep='&', 
              line_terminator='\\\ ', quotechar=' '))

write_txt(text_dir / 'trade_date.txt', dtxt(dt)['mon1'])
write_txt(text_dir / 'trade_date2.txt', dtxt(dt2)['mon1'])

shval = (final['Total '].iloc[1:].sum() / 
         final['Total '].iloc[0]) * 100
text = (f'In {ltdate}, trade with the top 25 trading partners (see table) '+
        f'comprises {shval:.1f} percent of total US trade in goods.')
write_txt(text_dir / 'trade_partner_sh.txt', text)
print(text)

In June 2021, trade with the top 25 trading partners (see table) comprises 87.2 percent of total US trade in goods.


### Trade Levels, Latest in Detail

In [4]:
result = pd.DataFrame()
url0 = 'https://www.census.gov/econ/currentdata/export/csv'
url1 = '?programCode=FTD&timeSlotType=12&startYear=1992&endYear=2021&categoryCode=BOPGS&'
for flow in ['EXP', 'IMP']:
    url2 = f'dataTypeCode={flow}&geoLevelCode=US&adjusted=yes&errorData=no&internal=false'
    url = url0 + url1 + url2
    df = pd.read_csv(url, skiprows=6).set_index('Period').dropna()
    df.index = pd.to_datetime(df.index)
    result[flow] = df['Value']
    
clean = result.loc['2015':].div(1000)
clean.to_csv(data_dir / 'tradelt.csv')
write_txt(text_dir / 'implt.txt', end_node3(clean['IMP'], 'blue!80!violet', percent=False))
write_txt(text_dir / 'explt.txt', end_node3(clean['EXP'], 'green!80!blue', percent=False))

In [5]:
ltdt = clean.index[-1]
ltdate = dtxt(ltdt)['mon1']
yrdate = dtxt(clean.index[-13])['mon1']
maydt = 'May' if ltdt.year == 2020 else 'May 2020'
imltval = clean.IMP.iloc[-1]
immayval = clean.IMP.loc['2020-05-01']
imyrval = clean.IMP.iloc[-13]
imp_comp = compare_text(imltval, imyrval, [10, 30, 50])
exltval = clean.EXP.iloc[-1]
exmayval = clean.EXP.loc['2020-05-01']
exyrval = clean.EXP.iloc[-13]
tbltval = exltval - imltval
tbmayval = exmayval - immayval
tbyrval = exyrval - imyrval

if (tbltval < 0) & (tbmayval < 0) & (tbyrval < 0):
    tbtxt = 'deficit'
    tbtxt2 = ''
    tbtxt3 = ''
else:
    tbtxt = 'deficit'
    tbtxt2 = 'a deficit of '
    tbtxt3 = 'a deficit of '
    if tbltval > 0:
        tbtxt = 'surplus'
    if tbmayval > 0:
        tbtxt2 = 'a surplus of '
    if tbyrval > 0:
        tbtxt3 = 'a surplus of '

text = (f'US imports of goods and services total \${imltval:.1f} billion in {ltdate}, '+
        f'compared to \${immayval:.1f} billion in {maydt}, and {imp_comp} the year ago value, '+
        f'\${imyrval:.1f} billion. Exports of goods and services are \${exltval:.1f} billion in '+
        f'{ltdate}, \${exmayval:.1f} billion in {maydt}, and \${exyrval:.1f} billion in {yrdate}. '+
        f'The trade balance, exports minus imports, shows a trade {tbtxt} of '+
        f'\${abs(tbltval):.1f} billion in {ltdate}, {tbtxt2}\${abs(tbmayval):.1f} billion '+
        f'in {maydt}, and {tbtxt3}\${abs(tbyrval):.1f} billion in {yrdate}. '+
        'These values are seasonally adjusted but not adjusted for changes in prices '+
        'or population. ')
write_txt(text_dir / 'tradeltlevels.txt', text)
print(text)

US imports of goods and services total \$283.4 billion in June 2021, compared to \$201.0 billion in May 2020, and far above the year ago value, \$209.5 billion. Exports of goods and services are \$207.7 billion in June 2021, \$146.1 billion in May 2020, and \$158.8 billion in June 2020. The trade balance, exports minus imports, shows a trade deficit of \$75.7 billion in June 2021, \$54.9 billion in May 2020, and \$50.7 billion in June 2020. These values are seasonally adjusted but not adjusted for changes in prices or population. 


### World Bank Tariff Data (WDI)

In [6]:
from io import BytesIO
from zipfile import ZipFile

url = 'http://api.worldbank.org/v2/en/indicator/TM.TAX.MRCH.WM.AR.ZS?downloadformat=csv'

r = requests.get(url)
zip_file = ZipFile(BytesIO(r.content))
files = zip_file.namelist()
data_file = [file for file in files if file[0] != 'M']
with zip_file.open(data_file[0], 'r') as csvfile:
    df = pd.read_csv(csvfile, skiprows=4)
    
data = df[df['Country Code'] == 'USA'].T.loc['1989':].dropna().iloc[:,0]
data.index = pd.to_datetime([f'{year}-07-01' for year in data.index])
data.name = 'tariff'
data.index.name = 'date'
data[pd.to_datetime('1994-07-01')] = np.nan
data = data.sort_index()
data.to_csv(data_dir / 'tariff.csv')
write_txt(text_dir / 'tariff_node.txt', end_node3(data, 'violet'))

t89 = data.loc['1989-07-01']
tlt = data.iloc[-1]
ltyear = dtxt(data.index[-1])['year']

text = ('World Bank staff \href{https://data.worldbank.org/indicator/TM.TAX.MRCH.WM.AR.ZS?locations=US&name_desc=true}{calculate} '+
        'the weighted average applied tariff rate on US imports. The measure shows the trend '+
        'in applied tariffs, factoring in the actual composition of imports by trading partner. '+
        f'In 1989, the average tariff on US imports was {t89:.1f} percent. During the 1990s and 2000s, '+
        'the US entered into various trade agreements, while China joined the WTO, obtaining '+
        '\href{https://www.wto.org/english/thewto_e/whatis_e/tif_e/fact2_e.htm}{most-favored nation} '+
        f'status with the US. In the latest data, covering {ltyear}, the average tariff was {tlt:.1f} '+
        'percent.')
write_txt(text_dir / 'tariff.txt', text)
print(text)

World Bank staff \href{https://data.worldbank.org/indicator/TM.TAX.MRCH.WM.AR.ZS?locations=US&name_desc=true}{calculate} the weighted average applied tariff rate on US imports. The measure shows the trend in applied tariffs, factoring in the actual composition of imports by trading partner. In 1989, the average tariff on US imports was 3.9 percent. During the 1990s and 2000s, the US entered into various trade agreements, while China joined the WTO, obtaining \href{https://www.wto.org/english/thewto_e/whatis_e/tif_e/fact2_e.htm}{most-favored nation} status with the US. In the latest data, covering 2019, the average tariff was 13.8 percent.
